# Investigate a Dataset: Titanic Data

## 1. Introduction

In this report, I will investigate the Titanic survivor data using exploratory data analysis.

In the data wrangling phase, I will determine the appropriate datatypes for our dataset, and I will also show how to handle missing values. 

In the data exploration phase, I will first look at each variable and its distribution. After that, I will answer two question:

1. What factors make people more likely to survive?

2. What money can buy? -- Explore relations among passenger class, cabin, and fare. 

Last, I will conclude this report by summarizing the findings and stating the limitations of my analysis.